In [155]:
# libraries
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report, roc_curve, auc)

import plotly.figure_factory as ff
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import LearningRateScheduler


import numpy as np

In [156]:
labels = ['alert', 'slight_drowsy','mod_drowsy','very_drowsy'] # alert, slightly drowsy, moderately drowsy and very drowsy

In [157]:
# Load the CSV file into a DataFrame
train_df = pd.read_csv('data/features/data_features_train_sampled_filt.csv')
val_df = pd.read_csv('data/features/data_features_val_sampled_filt.csv')
test_df = pd.read_csv('data/features/data_features_test_sampled_filt.csv')

dfs = {'train': train_df, 'val': val_df, 'test': test_df}
dfs_encoded = {}

for name, df in dfs.items():
  # Perform one-hot encoding using get_dummies()
  encoded_columns = pd.get_dummies(df['target'])

  # Reorder the columns of encoded_columns to match the desired order
  encoded_columns = encoded_columns[labels]

  # Concatenate the one-hot encoded columns with the original DataFrame
  df_encoded = pd.concat([df, encoded_columns], axis=1)

  dfs_encoded[name] = df_encoded

train_df = dfs_encoded['train']
val_df = dfs_encoded['val']
test_df = dfs_encoded['test']


In [158]:
print(train_df.shape,val_df.shape,test_df.shape)

(1384, 12) (296, 12) (295, 12)


In [159]:
train_df.head()

,blinks,right_EAR,left_EAR,blink_duration,blink_frequency,MAR,image,target,alert,slight_drowsy,mod_drowsy,very_drowsy
0,0,0.430978,0.460325,0.116667,12.000000,0.124741,alert_3_1603.png,alert,1,0,0,0
1,0,0.542564,0.518779,0.066667,12.000000,0.057461,alert_3_1031.png,alert,1,0,0,0
2,0,0.591392,0.602394,0.057747,13.800623,0.181665,alert_1_1023.png,alert,1,0,0,0
3,0,0.471022,0.514935,0.066667,6.000000,0.096739,alert_3_711.png,alert,1,0,0,0
4,0,0.527144,0.576907,0.033333,6.000000,0.116138,alert_1_743.png,alert,1,0,0,0


In [160]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns you want to scale
columns_to_scale = ["right_EAR", "left_EAR", "blink_duration", "blink_frequency", "MAR"]
# Scale the selected columns
train_df[columns_to_scale] = scaler.fit_transform(train_df[columns_to_scale])
test_df[columns_to_scale] = scaler.transform(test_df[columns_to_scale])
val_df[columns_to_scale] = scaler.transform(val_df[columns_to_scale])

In [161]:
train_df.describe()

,blinks,right_EAR,left_EAR,blink_duration,blink_frequency,MAR,alert,slight_drowsy,mod_drowsy,very_drowsy
count,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000,1384.000000
mean,0.061416,0.708216,0.691064,0.089053,0.232683,0.113580,0.252890,0.251445,0.244220,0.251445
std,0.240179,0.129790,0.125923,0.098384,0.211322,0.127889,0.434826,0.434001,0.429779,0.434001
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.657022,0.635815,0.030517,0.059096,0.047663,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.728607,0.708574,0.062500,0.142137,0.071420,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.792584,0.771821,0.127537,0.363636,0.123081,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [162]:
train_df['target'].value_counts()

alert            350
slight_drowsy    348
very_drowsy      348
mod_drowsy       338
Name: target, dtype: int64

In [163]:
# Save the train, test, and validation DataFrames to separate CSV files
train_df.to_csv('data/features/processed/train_features.csv', index=False)
test_df.to_csv('data/features/processed/test_features.csv', index=False)
val_df.to_csv('data/features/processed/val_features.csv', index=False)

In [5]:
# Save the train, test, and validation DataFrames to separate CSV files
train_df = pd.read_csv('data/features/processed/train_features.csv')
test_df = pd.read_csv('data/features/processed/test_features.csv')
val_df = pd.read_csv('data/features/processed/val_features.csv')

In [164]:
# Create ImageDataGenerators for train, test, and validation sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values
val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values

# Create train, test, and validation image generators using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='data/images/sample/train',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(224, 224),
    class_mode='raw',
    shuffle=True,
    seed=1
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='data/images/sample/test',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(224, 224),
    class_mode='raw',
    shuffle=False,
    seed=1
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='data/images/sample/val',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(224, 224),
    class_mode='raw',
    shuffle=False,
    seed=1
)


Found 1384 validated image filenames.
Found 295 validated image filenames.
Found 296 validated image filenames.


In [29]:
X,y = train_generator.next()
print(X.shape, y.shape)

(32, 224, 224, 3) (32, 4)


In [18]:
train_df.drop(columns=['image','target']+labels).columns.tolist()

['blinks', 'right_EAR', 'left_EAR', 'blink_duration', 'blink_frequency', 'MAR']

In [165]:
def custom_generator(image_generator, feature_data, batch_size):
    feature_index = 0
    total_samples = len(feature_data)
    while True:
        batch_images, batch_labels = image_generator.next()
        batch_features = feature_data[feature_index : feature_index + batch_size].values
        if (feature_index + batch_size) > total_samples:
            feature_index = 0
        else:
            feature_index = feature_index + batch_size

        yield ([batch_images, batch_features], batch_labels)

# Create custom generators combining image and feature data
train_generator_with_features = custom_generator(train_generator, train_df.drop(columns=['image','target']+labels), batch_size=32)
test_generator_with_features = custom_generator(test_generator, test_df.drop(columns=['image','target']+labels), batch_size=32)
val_generator_with_features = custom_generator(val_generator, val_df.drop(columns=['image','target']+labels), batch_size=32)


In [30]:
[x1,x2],y=next(test_generator_with_features)
print(x1.shape, x2.shape, y.shape)

(32, 224, 224, 3) (32, 6) (32, 4)


In [179]:
# # Load the pre-trained VGG16 model without the top layers
# vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# vgg16.trainable = False # Set all layers in the VGG16 model as non-trainable

# # Define the image input branch
# image_input = Input(shape=(224, 224, 3)) #(720, 1280, 3)
# vgg_output = vgg16(image_input)
# conv1 = Conv2D(32, (3, 3), activation='relu')(vgg_output)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
# # Add more convolutional and pooling layers as needed

# # Define the numerical feature input branch
# feature_input = Input(shape=(6,))
# dense1 = Dense(64, activation='relu')(feature_input)
# # Add more dense layers as needed

# # Concatenate the outputs from both branches
# concatenated = keras.layers.concatenate([Flatten()(pool1), dense1])

# # Add a final dense layer for prediction
# output = Dense(4, activation='softmax')(concatenated)

# # Create the model
# model = Model(inputs=[image_input, feature_input], outputs=output)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Define the image input branch
image_input = Input(shape=(224, 224, 3)) #(720, 1280, 3)
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
bn1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)
dropout1 = Dropout(0.25)(pool1)
conv2 = Conv2D(16, (3, 3), activation='relu')(dropout1)
bn2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)
dropout2 = Dropout(0.25)(pool2)
conv3 = Conv2D(8, (3, 3), activation='relu')(dropout2)
bn3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)

# Define the numerical feature input branch
feature_input = Input(shape=(6,))
dense1 = Dense(64, activation='relu')(feature_input)
dropout3 = Dropout(0.25)(dense1)
dense2 = Dense(32, activation='relu')(dropout3)
dropout4 = Dropout(0.25)(dense2)
dense3 = Dense(16, activation='relu')(dropout4)

# Concatenate the outputs from both branches
concatenated = keras.layers.concatenate([Flatten()(pool3), dense3])

# Add a final dense layer for prediction
output = Dense(4, activation='softmax')(concatenated)

# Create the model
model = Model(inputs=[image_input, feature_input], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch):
    initial_lr = 0.001
    drop_rate = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop_rate ** (epoch // epochs_drop))
    return lr

scheduler = LearningRateScheduler(lr_scheduler)

In [166]:
# Define the image input branch
image_input = Input(shape=(224, 224, 3)) #(720, 1280, 3)
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
bn1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)
conv2 = Conv2D(16, (5, 5), activation='relu')(pool1)
bn2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)
conv3 = Conv2D(8, (8, 8), activation='relu')(pool2)
bn3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)

# Define the numerical feature input branch
feature_input = Input(shape=(6,))
dense1 = Dense(64, activation='relu')(feature_input)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(16, activation='relu')(dense2)

# Concatenate the outputs from both branches
concatenated = keras.layers.concatenate([Flatten()(pool3), dense3])

# Add a final dense layer for prediction
output = Dense(4, activation='softmax')(concatenated)

# Create the model
model = Model(inputs=[image_input, feature_input], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch):
    initial_lr = 0.001
    drop_rate = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop_rate ** (epoch // epochs_drop))
    return lr

scheduler = LearningRateScheduler(lr_scheduler)

In [167]:
# fit network
history = model.fit(train_generator_with_features,
                    epochs=20,
                    steps_per_epoch=len(train_generator),
                    validation_data=val_generator_with_features,
                    validation_steps= len(val_generator),
                    callbacks=[scheduler])

# 29 mins con 3 epochs => model_3
# 6 mins 34s solo he probado primero con 1 epoch => model_4
# de aquí para abajo con vscode
# 34.1s, 1 epoch, en vscode en vez de google que tarda por tener que ir a drive => model_5
# 8 mins y pico, 1 epoch, con vgg16 => pero ha fallado en la validación al concatenar, estaba tardando mucho tampoco lo iba a usar

# en todo lo de arriba estaban mal los generators, por eso ha fallado el vgg16

# 13 mins 28.8s, ordenador sobrecalentado (por eso tarda), 5 epochs, modelo con más capas => model_6

# 22 mins 46.5s, sigue caliente, 10 epochs y poniendo el epochs drop a 5 que antes no hacía nada, modelo del 6, shuffle true en train_gen => model_7
# al hacer el shuffle ya consigue aprender de los datos de train, pero hay que poner dropout porque en validation va mal
# no parece que el lr esté cambiando, vale es que no lo había ejecutado con el cambio
# mierda he borrado el 7 sin querer, (el que pone 7 es el 8 del model)

# 28 mins 11.6s, sigue caliente, 10 epochs, con el epoch drop a 5 ahora sí, añadiendo dropout => model_8
# el dropout no parece que haga mucho, estoy llegando a accuracy 1 en train y val sigue bajo

# 10 mins solo 1 epoch, sigue caliente, vgg16, 4 epochs por probar, con los arreglos de los generators y el shuffle que antes igual no aprendía porque estaba mal => model_9
# ETA del primero ya son como 9 mins, no lo voy a dejar hasta el final, solo quiero ver el val_accuracy como es

# 4 mins 50.8s, 4 epochs, con las nuevas fotos, sin dropout => model_10

# 6 mins 13s, ordenador de Fer, 15 epochs, con las nuevas fotos, sin dropout => model_11
# 6 mins 37.3s, igual que el 11 pero probando con dropout => model_12
# el tiempo ya será parecido, igual que el 11 (sin dropout), he cambiado los kernel => model_13
# 11 mins 44.8s, el 13 pero con 20 epochs => model_14
# 11 mins 27.3s, metiendo datos de slight_3 último minuto en train => model_15

Epoch 1/20
44/44 [==============================] - 38s 822ms/step - loss: 0.5165 - accuracy: 0.8201 - val_loss: 1.3548 - val_accuracy: 0.5236 - lr: 0.0010
Epoch 2/20
44/44 [==============================] - 34s 781ms/step - loss: 0.1084 - accuracy: 0.9603 - val_loss: 0.9747 - val_accuracy: 0.5405 - lr: 0.0010
Epoch 3/20
44/44 [==============================] - 34s 783ms/step - loss: 0.1135 - accuracy: 0.9632 - val_loss: 1.3017 - val_accuracy: 0.4020 - lr: 0.0010
Epoch 4/20
44/44 [==============================] - 35s 785ms/step - loss: 0.0464 - accuracy: 0.9841 - val_loss: 1.4653 - val_accuracy: 0.3615 - lr: 0.0010
Epoch 5/20
44/44 [==============================] - 34s 781ms/step - loss: 0.0197 - accuracy: 0.9942 - val_loss: 1.5205 - val_accuracy: 0.3750 - lr: 0.0010
Epoch 6/20
44/44 [==============================] - 34s 782ms/step - loss: 0.0138 - accuracy: 0.9971 - val_loss: 1.4455 - val_accuracy: 0.3784 - lr: 5.0000e-04
Epoch 7/20
44/44 [==============================] - 34s 779m

In [168]:
model.save('models/model_15.h5')

In [169]:
np.save('history_15.npy', history.history)

In [60]:
# model = load_model('models/model_13.h5') # model 1 y 2 primeros (1 sin val y 2 con val), con los 9000 y val/test acc 1, epoch 1
# model 3 es con el sample y 3 epochs

In [170]:
prediction = model.predict(test_generator_with_features, steps=len(test_generator))
# 4 min 56s => model_3
# 2s => model_4
# 2.6s => model_5
# 13.9s => model_6 (ordenador sobrecalentado no sé si afectará)
# 8.4s => model_7  (sigue caliente)
# 3.3s => model_8 (sigue caliente)
# 5.7s => model_10
# 1.5s => model_11 (Fer)

10/10 [==============================] - 2s 146ms/step


In [171]:
test_pred = np.argmax(prediction, axis=1)
test_pred.shape

(295,)

In [172]:
test_true_values = test_df[labels].to_numpy()
test_true = np.argmax(test_true_values, axis=1)
test_true.shape

(295,)

In [174]:
accuracy = accuracy_score(test_true, test_pred)
accuracy
# model_3
# en test sigue teniendo un 100% accuracy ?????
# añadir dropout o regularización?
# igual sigue siendo demasiado parecido, durante el mes ir cargando el resto de mins de los vídeos
# intentar liar a alguien para grabarles también

# model_4
# ya da un accuracy malo como quería
# añadir más capas a la red (maybe vgg16)
# y también más epochs que solo he puesto 1
# 0.18 o así

# model_6
# 0.284

# model_7
# 0.3545

# model_8
# 0.197

# model_11
# 0.7726 por fin algo decente

# model_12
# 0.6254 el dropout es demasiado

# model_13
# 0.8127090301003345 => los generator algo va mal, se ve que no se está reiniciando bien

# model_15
# 0.9220338983050848

0.9220338983050848

In [175]:
print(classification_report(test_true,test_pred, target_names=labels, digits=4))


               precision    recall  f1-score   support

        alert     0.9615    1.0000    0.9804        75
slight_drowsy     0.9857    0.9452    0.9650        73
   mod_drowsy     0.7865    0.9722    0.8696        72
  very_drowsy     1.0000    0.7733    0.8722        75

     accuracy                         0.9220       295
    macro avg     0.9334    0.9227    0.9218       295
 weighted avg     0.9346    0.9220    0.9220       295



In [176]:
# Construct a visualization for the confusion matrix
z_test = confusion_matrix(test_true,test_pred)

# reformat the matrix
z_test[[0,1,2,3],:] = z_test[[3,2,1,0],:]

x = ['alert','slight_drowsy','mod_drowsy','very_drowsy']
y = ['very_drowsy','mod_drowsy', 'slight_drowsy', 'alert']

def format_text(z):
    """
    This function receives the confusion matrix values and formats the text
    to display in the matrix with the value and percentage of total data.
    """
    return f'{z}<br>({z / np.sum(z_test) * 100:.1f}%)'

z_text = np.vectorize(format_text)(z_test)

heatmap = ff.create_annotated_heatmap(z_test, x=x, y=y, annotation_text=z_text, colorscale='plasma')
heatmap.update_layout(height=350,width=500,
                      xaxis_title="Predicted Label",yaxis_title="True Label")
heatmap.show()

In [178]:
cont=0
for i in range(295):
    if test_true[i]==3 and test_pred[i]==2:
        cont+=1
        print(test_df.iloc[i]['image'])

very_drowsy_2_596.png
very_drowsy_2_866.png
very_drowsy_2_558.png
very_drowsy_2_893.png
very_drowsy_2_506.png
very_drowsy_2_590.png
very_drowsy_2_589.png
very_drowsy_2_757.png
very_drowsy_2_761.png
very_drowsy_2_870.png
very_drowsy_2_843.png
very_drowsy_2_524.png
very_drowsy_2_768.png
very_drowsy_2_764.png
very_drowsy_2_860.png
very_drowsy_2_534.png
